<a href="https://colab.research.google.com/github/ryanhao1115/ML-for-Fraud-Detection/blob/main/5_1_DL_ordinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Deep Learning model with Ordinal encoding data
1. import new return label data.
2. Under sampling data to address imbalance class issue.
4. Train model
5. Eval model
6. Create Risk-heatmap

In [138]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.backend import clear_session
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import tensorflow as tf
import random


In [77]:
def reset_seeds():
    '''
    reset random seeds for modeling
    '''
    np.random.seed(1)
    random.seed(2)
    if tf.__version__[0] == '2':
        tf.random.set_seed(3)
    else:
        tf.set_random_seed(3)
    print("RANDOM SEEDS RESET")

In [5]:
## import dataset 
path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
file = path + 'sales_clean.csv'
df = pd.read_csv(file)

In [6]:
df = df.drop(columns='Unnamed: 0')

In [7]:
df.nunique()

distributor        639
sales              209
branch              28
inv_type             3
invoice_no       30721
product_no        1094
prod_cla             7
qty                202
total_amt        16112
sale_price        8598
ship_qty           202
cust_type           20
return               2
discount_app         2
list_price        6720
inv_ship_days      133
dtype: int64

## import return label data

In [8]:
## import fraud data
file = path + 'frauds.csv'
df_fraud = pd.read_csv(file,header=None)
df_fraud.columns = ['invoice_no']

In [9]:
## import return new labeled data
file = path + 'return.csv'
df_fraud2 = pd.read_csv(file,header=None)
df_fraud2.columns = ['invoice_no']

In [10]:
len(df_fraud2)

24

In [11]:
def label_fraud(df, df_fraud, df_fraud2):
  df['fraud'] = np.zeros(len(df))
  frauds_l = df_fraud['invoice_no'].to_list()
  frauds_l2 = df_fraud2['invoice_no'].to_list()
  df.loc[df['invoice_no'].isin(frauds_l),'fraud'] = 1
  df.loc[df['invoice_no'].isin(frauds_l2),'fraud'] = 2
  return df

In [12]:
df = label_fraud(df, df_fraud,df_fraud2)

In [13]:
df['fraud'].value_counts()

0.0    228014
2.0       433
1.0       154
Name: fraud, dtype: int64

## Resampling to address the imbalance class
Because fraud labels are given to invoice level, need to keep records of same invoice together. 

In [20]:
df_fraud = df[df['fraud'] == 1]
df_non_fraud = df[df['fraud'] == 0]
df_fraud2 = df[df['fraud'] == 2]

In [21]:
df_fraud2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433 entries, 19646 to 217825
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   distributor    433 non-null    object 
 1   sales          433 non-null    object 
 2   branch         433 non-null    int64  
 3   inv_type       433 non-null    object 
 4   invoice_no     433 non-null    int64  
 5   product_no     433 non-null    object 
 6   prod_cla       433 non-null    float64
 7   qty            433 non-null    int64  
 8   total_amt      433 non-null    float64
 9   sale_price     433 non-null    float64
 10  ship_qty       433 non-null    int64  
 11  cust_type      433 non-null    object 
 12  return         433 non-null    int64  
 13  discount_app   433 non-null    int64  
 14  list_price     433 non-null    float64
 15  inv_ship_days  433 non-null    int64  
 16  fraud          433 non-null    float64
dtypes: float64(5), int64(7), object(5)
memory usage

In [27]:
def sampling_inv(df_non_fraud):
  '''
  Random sampling from unlabled invoices.
  '''
  sample_inv = df_non_fraud['invoice_no'].unique()
  print(len(sample_inv))
  sample_inv1 = np.random.choice(sample_inv,size=1000,replace=False)
  sample_inv = np.setdiff1d(sample_inv,sample_inv1)
  print(len(sample_inv))
  sample_inv2 = np.random.choice(sample_inv,size=5000,replace=False)
  sample_inv = np.setdiff1d(sample_inv,sample_inv2)
  print(len(sample_inv))
  return sample_inv1, sample_inv2, sample_inv


In [28]:
sample_inv1, sample_inv2, sample_inv = sampling_inv(df_non_fraud)

30686
29686
24686


In [30]:
len(np.unique(sample_inv2))

5000

In [31]:
## Build training dataset
df_train = df_non_fraud[df_non_fraud['invoice_no'].isin(sample_inv2)]
df_train = pd.concat([df_train,df_fraud2])
df_train.loc[df_train['fraud'] == 2, 'fraud'] = 1 
df_train = df_train.sort_index()

In [35]:
## Build eval dataset
df_test = df_non_fraud[df_non_fraud['invoice_no'].isin(sample_inv1)]
df_test = pd.concat([df_test,df_fraud])
df_test = df_test.sort_index()

In [33]:
df_train.shape

(37232, 17)

In [36]:
df_test.shape

(7465, 17)

In [37]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37232 entries, 108 to 228534
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   distributor    37232 non-null  object 
 1   sales          37232 non-null  object 
 2   branch         37232 non-null  int64  
 3   inv_type       37232 non-null  object 
 4   invoice_no     37232 non-null  int64  
 5   product_no     37232 non-null  object 
 6   prod_cla       37232 non-null  float64
 7   qty            37232 non-null  int64  
 8   total_amt      37232 non-null  float64
 9   sale_price     37232 non-null  float64
 10  ship_qty       37232 non-null  int64  
 11  cust_type      37232 non-null  object 
 12  return         37232 non-null  int64  
 13  discount_app   37232 non-null  int64  
 14  list_price     37232 non-null  float64
 15  inv_ship_days  37232 non-null  int64  
 16  fraud          37232 non-null  float64
dtypes: float64(5), int64(7), object(5)
memory usage

## Encoding

In [38]:
cols = df.columns.to_list()
cols = ['distributor', 'sales', 'branch', 'inv_type', 'invoice_no', 'product_no', 'prod_cla', 'cust_type']

In [39]:
def field_encoding(df_train, df_eval):
  '''
  Ordinal encode categorical fields.
  '''
  cols = ['distributor', 'sales', 'branch', 'inv_type', 'invoice_no', 'product_no', 'prod_cla', 'cust_type']
  df_train[cols] = df_train[cols].astype('str') 
  df_test[cols] = df_test[cols].astype('str') 
  df1 = pd.concat([df_train, df_test])
  enc = LabelEncoder()
  for f in cols:
    enc.fit(df1[f])
    df_train[f] = enc.transform(df_train[f])
    df_test[f] = enc.transform(df_test[f])
  return df_train, df_test

In [47]:
X_train = df_train.copy()
X_test = df_test.copy()
X_train, X_test = field_encoding(X_train, X_test)

In [110]:
print(len(X_test),len(X_train))

7465 37232


In [41]:
X_train.nunique()

distributor       518
sales             187
branch             28
inv_type            3
invoice_no       5024
product_no        785
prod_cla            7
qty                89
total_amt        5363
sale_price       3725
ship_qty           90
cust_type          20
return              2
discount_app        2
list_price       3012
inv_ship_days      75
fraud               2
dtype: int64

## Standardscale data

In [45]:
scaler = StandardScaler()

In [48]:
y_train = X_train['fraud'].values
X_train = X_train.drop(columns=['fraud'])

X_train = scaler.fit_transform(X_train)
y_train = y_train.reshape(len(y_train),1)

In [49]:
y_test = X_test['fraud'].values
X_test = X_test.drop(columns=['fraud'])

X_test = scaler.transform(X_test)
y_test = y_test.reshape(len(y_test),1)

In [50]:
X_train[0]

array([ 0.31969424,  0.81761482, -1.18807636,  0.01829786, -1.70644042,
        1.7724664 , -0.0709056 , -0.00639322,  1.28657546,  3.27756406,
        0.0045258 , -0.77624262, -0.0163908 ,  0.01036562,  3.27549634,
       -0.06245374])

In [51]:
y_test.shape

(7465, 1)

In [60]:
y_train.sum()

433.0

## Build model

In [199]:
# define the  model
def build_model():
  model = Sequential()
  model.add(Dense(32, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
  model.add(Dense(16,activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # compile the keras model
  # opt = tf.keras.optimizers.Adam(clipnorm=1, learning_rate=0.01)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [109]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1088      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 3,201
Trainable params: 3,201
Non-trainable params: 0
_________________________________________________________________


## Training model

In [213]:
## clear previous model.  Make sure training from 0. 
del model
clear_session()
tf.compat.v1.reset_default_graph()
reset_seeds()
model = build_model()

RANDOM SEEDS RESET


In [214]:
model.fit(X_test, y_test, epochs=500,batch_size=64,class_weight={0:5, 1:300})

Epoch 1/500
117/117 [==============================] - 1s 2ms/step - loss: 9.3178 - accuracy: 0.4761
Epoch 2/500
117/117 [==============================] - 0s 2ms/step - loss: 7.8343 - accuracy: 0.5102
Epoch 3/500
117/117 [==============================] - 0s 2ms/step - loss: 6.8232 - accuracy: 0.6276
Epoch 4/500
117/117 [==============================] - 0s 2ms/step - loss: 7.2422 - accuracy: 0.5566
Epoch 5/500
117/117 [==============================] - 0s 2ms/step - loss: 6.5429 - accuracy: 0.6435
Epoch 6/500
117/117 [==============================] - 0s 2ms/step - loss: 6.3730 - accuracy: 0.6548
Epoch 7/500
117/117 [==============================] - 0s 2ms/step - loss: 6.4800 - accuracy: 0.6221
Epoch 8/500
117/117 [==============================] - 0s 2ms/step - loss: 6.2976 - accuracy: 0.6583
Epoch 9/500
117/117 [==============================] - 0s 2ms/step - loss: 6.5031 - accuracy: 0.6643
Epoch 10/500
117/117 [==============================] - 0s 2ms/step - loss: 5.9343 - accura

In [195]:
loss,accuracy = model.evaluate(X_test, y_test)
print(loss, accuracy)

234/234 [==============================] - 0s 1ms/step - loss: 8.6929 - accuracy: 0.9794
8.692902565002441 0.9793704152107239


## Predict on eval dataset for further labeling.

In [215]:
y_pred = model.predict(X_train)
y_cls = (y_pred > 0.5).astype("int32")

In [60]:
(y_pred > 0.5).sum()

827

In [116]:
def eval_model(y_test, y_cls):
  y1 = y_test.reshape(-1)
  y2 = y_cls.reshape(-1)
  print(roc_auc_score(y1, y2))
  print(classification_report(y1,y2))
  print(confusion_matrix(y1,y2))


In [216]:
eval_model(y_train, y_cls)

0.4913836899499039
              precision    recall  f1-score   support

         0.0       0.99      0.29      0.45     36799
         1.0       0.01      0.69      0.02       433

    accuracy                           0.30     37232
   macro avg       0.50      0.49      0.24     37232
weighted avg       0.98      0.30      0.45     37232

[[10754 26045]
 [  134   299]]


In [203]:
eval_model(y_train, y_cls)

0.5064665001502765
              precision    recall  f1-score   support

         0.0       0.99      0.67      0.80     36799
         1.0       0.01      0.35      0.02       433

    accuracy                           0.66     37232
   macro avg       0.50      0.51      0.41     37232
weighted avg       0.98      0.66      0.79     37232

[[24527 12272]
 [  283   150]]


In [197]:
eval_model(y_test, y_cls)

0.5
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      7311
         1.0       0.00      0.00      0.00       154

    accuracy                           0.98      7465
   macro avg       0.49      0.50      0.49      7465
weighted avg       0.96      0.98      0.97      7465

[[7311    0]
 [ 154    0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [84]:
def export_result(df_eval,y_pred):
  '''
  Attach the predicte result (probability) into original records.
  Export to a csv file for further investication
  '''
  df_eval['predict'] = y_pred
  df_eval['mid_value'] = abs(df_eval['predict'] - 0.5)
  df_eval = df_eval.sort_values('mid_value')
  value_limit = df_eval.iloc[200,-1]
  df_record4label = df_eval[(df_eval['mid_value'] <= value_limit)|(df_eval['fraud'] == 1)]
  path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
  file_csv = path + 'for_label.csv'
  df_record4label.to_csv(file_csv)
  return True


In [85]:
record_to_label(df_eval,y_pred)

True